In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
from pylab import rcParams
from collections import Counter
import copy

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pickle
from sklearn.metrics import mean_absolute_error
torch.manual_seed(1)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device) #Google colab = TESLA T4
torch.cuda.empty_cache()

In [ ]:
# 1분간격으로 데이터를 취득했기 때문에, Time step을 60(초)로 지정해주어 데이터를 처리해줍니다.
TIME_STEP = 60 # 60초 (1분)

#빈 리스트를 먼저 만들어 주어 4,5,6번 after 데이터셋에서 vdiff(전압편차), curr(전류), temp(온도), soc(soc) 변수만 가져와줍니다.

train_list = []

for (vin_num,case) in [[4,'AFTER'],[5,'AFTER'],[6,'AFTER']]:
    
    df__ = pd.read_csv(f"./vin_{vin_num:02d}_2021_{case}_SORT.csv")
    
    df__['vdiff']     = pd.DataFrame(df__.apply(lambda x : (x['bms_26_500ms_2_xev_bms_vmax_dec']-x['bms_26_500ms_2_xev_bms_vmin_dec']), axis = 1 ))   
    df__['curr']      = pd.DataFrame(df__.apply(lambda x :  x['bms_02_100ms_2_xev_battcurrval'], axis = 1 ))       
    df__['temp']      = pd.DataFrame(df__.apply(lambda x : (x['bms_02_100ms_2_xev_maxbatttempval']+x['bms_02_100ms_2_xev_minbatttempval'])/2, axis = 1 ))       
    df__['soc']       = pd.DataFrame(df__.apply(lambda x : (x['bms_01_100ms_2_xev_socval']+x['bms_01_100ms_2_xev_socdecval']), axis = 1 ))       
    
    num_trip = len(df__['index'].unique())
    
    print(f'vin: {vin_num} / # of trip: {num_trip}')
    
    for k in df__['index'].unique():
    
        df_trip = df__.loc[df__['index']==k,:].drop_duplicates().reset_index(drop = True)
    
        idx_trip = df_trip['index'].iloc[0]
        len_trip = len(df_trip)
        
        print(f'idx_trip: {idx_trip} / len_trip: {len_trip}')
        
        
        if len(df_trip) > TIME_STEP:
            
            for i in range(len(df_trip)-TIME_STEP+1):
                window = df_trip.iloc[i:(i+TIME_STEP),:]
                window = window[['vdiff','curr', 'temp', 'soc']]
                window = window.values.tolist()
                train_list.append(window)
               

In [ ]:
train_list4_2 = []
for (vin_num,case) in [[4,'AFTER']]:
    
    df__ = pd.read_csv(f"./vin_{vin_num:02d}_2021_{case}_SORT.csv")
    
    df__['vdiff']     = pd.DataFrame(df__.apply(lambda x : (x['bms_26_500ms_2_xev_bms_vmax_dec']-x['bms_26_500ms_2_xev_bms_vmin_dec']), axis = 1 ))   
    df__['curr']      = pd.DataFrame(df__.apply(lambda x :  x['bms_02_100ms_2_xev_battcurrval'], axis = 1 ))       
    df__['temp']      = pd.DataFrame(df__.apply(lambda x : (x['bms_02_100ms_2_xev_maxbatttempval']+x['bms_02_100ms_2_xev_minbatttempval'])/2, axis = 1 ))       
    df__['soc']       = pd.DataFrame(df__.apply(lambda x : (x['bms_01_100ms_2_xev_socval']+x['bms_01_100ms_2_xev_socdecval']), axis = 1 ))       
    
    num_trip = len(df__['index'].unique())
    
    print(f'vin: {vin_num} / # of trip: {num_trip}')
    
    for k in df__['index'].unique():
    
        df_trip = df__.loc[df__['index']==k,:].drop_duplicates().reset_index(drop = True)
    
        idx_trip = df_trip['index'].iloc[0]
        len_trip = len(df_trip)
        
        print(f'idx_trip: {idx_trip} / len_trip: {len_trip}')
        
        
        if len(df_trip) > TIME_STEP:
            
            for i in range(len(df_trip)-TIME_STEP+1):
                window = df_trip.iloc[i:(i+TIME_STEP),:]
                window = window[['vdiff','curr', 'temp', 'soc']]
                window = window.values.tolist()
                train_list4_2.append(window)
                 

In [ ]:
train_list5_2 = []
for (vin_num,case) in [[5,'AFTER']]:
    
    df__ = pd.read_csv(f"./vin_{vin_num:02d}_2021_{case}_SORT.csv")
    
    df__['vdiff']     = pd.DataFrame(df__.apply(lambda x : (x['bms_26_500ms_2_xev_bms_vmax_dec']-x['bms_26_500ms_2_xev_bms_vmin_dec']), axis = 1 ))   
    df__['curr']      = pd.DataFrame(df__.apply(lambda x :  x['bms_02_100ms_2_xev_battcurrval'], axis = 1 ))       
    df__['temp']      = pd.DataFrame(df__.apply(lambda x : (x['bms_02_100ms_2_xev_maxbatttempval']+x['bms_02_100ms_2_xev_minbatttempval'])/2, axis = 1 ))       
    df__['soc']       = pd.DataFrame(df__.apply(lambda x : (x['bms_01_100ms_2_xev_socval']+x['bms_01_100ms_2_xev_socdecval']), axis = 1 ))       
    
    num_trip = len(df__['index'].unique())
    
    print(f'vin: {vin_num} / # of trip: {num_trip}')
    
    for k in df__['index'].unique():
    
        df_trip = df__.loc[df__['index']==k,:].drop_duplicates().reset_index(drop = True)
    
        idx_trip = df_trip['index'].iloc[0]
        len_trip = len(df_trip)
        
        print(f'idx_trip: {idx_trip} / len_trip: {len_trip}')
        
        
        if len(df_trip) > TIME_STEP:
            
            for i in range(len(df_trip)-TIME_STEP+1):
                window = df_trip.iloc[i:(i+TIME_STEP),:]
                window = window[['vdiff','curr', 'temp', 'soc']]
                window = window.values.tolist()
                train_list5_2.append(window)
                 

In [ ]:
train_list6_2 = []
for (vin_num,case) in [[6,'AFTER']]:
    
    df__ = pd.read_csv(f"./vin_{vin_num:02d}_2021_{case}_SORT.csv")
    
    df__['vdiff']     = pd.DataFrame(df__.apply(lambda x : (x['bms_26_500ms_2_xev_bms_vmax_dec']-x['bms_26_500ms_2_xev_bms_vmin_dec']), axis = 1 ))   
    df__['curr']      = pd.DataFrame(df__.apply(lambda x :  x['bms_02_100ms_2_xev_battcurrval'], axis = 1 ))       
    df__['temp']      = pd.DataFrame(df__.apply(lambda x : (x['bms_02_100ms_2_xev_maxbatttempval']+x['bms_02_100ms_2_xev_minbatttempval'])/2, axis = 1 ))       
    df__['soc']       = pd.DataFrame(df__.apply(lambda x : (x['bms_01_100ms_2_xev_socval']+x['bms_01_100ms_2_xev_socdecval']), axis = 1 ))       
    
    num_trip = len(df__['index'].unique())
    
    print(f'vin: {vin_num} / # of trip: {num_trip}')
    
    for k in df__['index'].unique():
    
        df_trip = df__.loc[df__['index']==k,:].drop_duplicates().reset_index(drop = True)
    
        idx_trip = df_trip['index'].iloc[0]
        len_trip = len(df_trip)
        
        print(f'idx_trip: {idx_trip} / len_trip: {len_trip}')
        
        
        if len(df_trip) > TIME_STEP:
            
            for i in range(len(df_trip)-TIME_STEP+1):
                window = df_trip.iloc[i:(i+TIME_STEP),:]
                window = window[['vdiff','curr', 'temp', 'soc']]
                window = window.values.tolist()
                train_list6_2.append(window)

In [ ]:
with open( "vin_04_2021_AFTER_SORT_data_list_data", "wb" ) as file:
    pickle.dump( train_list4_2, file)
with open( "vin_05_2021_AFTER_SORT_data_list_data", "wb" ) as file:
    pickle.dump( train_list5_2, file)   
with open( "vin_06_2021_AFTER_SORT_data_list_data", "wb" ) as file:
    pickle.dump( train_list6_2, file)

with open( "train_list", "wb" ) as file:
    pickle.dump( train_list, file)

In [ ]:
test_list1_1 = []
for (vin_num,case) in [[1,'BEFORE']]:
    
    df__ = pd.read_csv(f"./vin_{vin_num:02d}_2021_{case}_SORT.csv")
    
    df__['vdiff']     = pd.DataFrame(df__.apply(lambda x : (x['bms_26_500ms_2_xev_bms_vmax_dec']-x['bms_26_500ms_2_xev_bms_vmin_dec']), axis = 1 ))   
    df__['curr']      = pd.DataFrame(df__.apply(lambda x :  x['bms_02_100ms_2_xev_battcurrval'], axis = 1 ))       
    df__['temp']      = pd.DataFrame(df__.apply(lambda x : (x['bms_02_100ms_2_xev_maxbatttempval']+x['bms_02_100ms_2_xev_minbatttempval'])/2, axis = 1 ))       
    df__['soc']       = pd.DataFrame(df__.apply(lambda x : (x['bms_01_100ms_2_xev_socval']+x['bms_01_100ms_2_xev_socdecval']), axis = 1 ))       
    
    num_trip = len(df__['index'].unique())
    
    print(f'vin: {vin_num} / # of trip: {num_trip}')
    
    for k in df__['index'].unique():
    
        df_trip = df__.loc[df__['index']==k,:].drop_duplicates().reset_index(drop = True)
    
        idx_trip = df_trip['index'].iloc[0]
        len_trip = len(df_trip)
        
        print(f'idx_trip: {idx_trip} / len_trip: {len_trip}')
        
        
        if len(df_trip) > TIME_STEP:
            
            for i in range(len(df_trip)-TIME_STEP+1):
                window = df_trip.iloc[i:(i+TIME_STEP),:]
                window = window[['vdiff','curr', 'temp', 'soc']]
                window = window.values.tolist()

                test_list1_1.append(window)

In [ ]:
test_list1_2 = []
for (vin_num,case) in [[1,'AFTER']]:
    
    df__ = pd.read_csv(f"./vin_{vin_num:02d}_2021_{case}_SORT.csv")
    df__ = df__.loc[df__['index'] > 250,:].drop_duplicates().reset_index(drop = True)
    df__['vdiff']     = pd.DataFrame(df__.apply(lambda x : (x['bms_26_500ms_2_xev_bms_vmax_dec']-x['bms_26_500ms_2_xev_bms_vmin_dec']), axis = 1 ))   
    df__['curr']      = pd.DataFrame(df__.apply(lambda x :  x['bms_02_100ms_2_xev_battcurrval'], axis = 1 ))       
    df__['temp']      = pd.DataFrame(df__.apply(lambda x : (x['bms_02_100ms_2_xev_maxbatttempval']+x['bms_02_100ms_2_xev_minbatttempval'])/2, axis = 1 ))       
    df__['soc']       = pd.DataFrame(df__.apply(lambda x : (x['bms_01_100ms_2_xev_socval']+x['bms_01_100ms_2_xev_socdecval']), axis = 1 ))       
    
    num_trip = len(df__['index'].unique())
    
    print(f'vin: {vin_num} / # of trip: {num_trip}')
    
    for k in df__['index'].unique():
    
        df_trip = df__.loc[df__['index']==k,:].drop_duplicates().reset_index(drop = True)
    
        idx_trip = df_trip['index'].iloc[0]
        len_trip = len(df_trip)
        
        print(f'idx_trip: {idx_trip} / len_trip: {len_trip}')
        
        
        if len(df_trip) > TIME_STEP:
            
            for i in range(len(df_trip)-TIME_STEP+1):
                window = df_trip.iloc[i:(i+TIME_STEP),:]
                window = window[['vdiff','curr', 'temp', 'soc']]
                window = window.values.tolist()

                test_list1_2.append(window)

In [ ]:
test_list2_1 = []
for (vin_num,case) in [[2,'BEFORE']]:
    
    df__ = pd.read_csv(f"./vin_{vin_num:02d}_2021_{case}_SORT.csv")
    
    df__['vdiff']     = pd.DataFrame(df__.apply(lambda x : (x['bms_26_500ms_2_xev_bms_vmax_dec']-x['bms_26_500ms_2_xev_bms_vmin_dec']), axis = 1 ))   
    df__['curr']      = pd.DataFrame(df__.apply(lambda x :  x['bms_02_100ms_2_xev_battcurrval'], axis = 1 ))       
    df__['temp']      = pd.DataFrame(df__.apply(lambda x : (x['bms_02_100ms_2_xev_maxbatttempval']+x['bms_02_100ms_2_xev_minbatttempval'])/2, axis = 1 ))       
    df__['soc']       = pd.DataFrame(df__.apply(lambda x : (x['bms_01_100ms_2_xev_socval']+x['bms_01_100ms_2_xev_socdecval']), axis = 1 ))       
    
    num_trip = len(df__['index'].unique())
    
    print(f'vin: {vin_num} / # of trip: {num_trip}')
    
    for k in df__['index'].unique():
    
        df_trip = df__.loc[df__['index']==k,:].drop_duplicates().reset_index(drop = True)
    
        idx_trip = df_trip['index'].iloc[0]
        len_trip = len(df_trip)
        
        print(f'idx_trip: {idx_trip} / len_trip: {len_trip}')
        
        
        if len(df_trip) > TIME_STEP:
            
            for i in range(len(df_trip)-TIME_STEP+1):
                window = df_trip.iloc[i:(i+TIME_STEP),:]
                window = window[['vdiff','curr', 'temp', 'soc']]
                window = window.values.tolist()

                test_list2_1.append(window)

In [ ]:
test_list2_2 = []
for (vin_num,case) in [[2,'AFTER']]:
    
    df__ = pd.read_csv(f"./vin_{vin_num:02d}_2021_{case}_SORT.csv")
    
    df__['vdiff']     = pd.DataFrame(df__.apply(lambda x : (x['bms_26_500ms_2_xev_bms_vmax_dec']-x['bms_26_500ms_2_xev_bms_vmin_dec']), axis = 1 ))   
    df__['curr']      = pd.DataFrame(df__.apply(lambda x :  x['bms_02_100ms_2_xev_battcurrval'], axis = 1 ))       
    df__['temp']      = pd.DataFrame(df__.apply(lambda x : (x['bms_02_100ms_2_xev_maxbatttempval']+x['bms_02_100ms_2_xev_minbatttempval'])/2, axis = 1 ))       
    df__['soc']       = pd.DataFrame(df__.apply(lambda x : (x['bms_01_100ms_2_xev_socval']+x['bms_01_100ms_2_xev_socdecval']), axis = 1 ))       
    
    num_trip = len(df__['index'].unique())
    
    print(f'vin: {vin_num} / # of trip: {num_trip}')
    
    for k in df__['index'].unique():
    
        df_trip = df__.loc[df__['index']==k,:].drop_duplicates().reset_index(drop = True)
    
        idx_trip = df_trip['index'].iloc[0]
        len_trip = len(df_trip)
        
        print(f'idx_trip: {idx_trip} / len_trip: {len_trip}')
        
        
        if len(df_trip) > TIME_STEP:
            
            for i in range(len(df_trip)-TIME_STEP+1):
                window = df_trip.iloc[i:(i+TIME_STEP),:]
                window = window[['vdiff','curr', 'temp', 'soc']]
                window = window.values.tolist()

                test_list2_2.append(window)
                 

In [ ]:
test_list4_1 = []
for (vin_num,case) in [[4,'BEFORE']]:
    
    df__ = pd.read_csv(f"./vin_{vin_num:02d}_2021_{case}_SORT.csv")
    
    df__['vdiff']     = pd.DataFrame(df__.apply(lambda x : (x['bms_26_500ms_2_xev_bms_vmax_dec']-x['bms_26_500ms_2_xev_bms_vmin_dec']), axis = 1 ))   
    df__['curr']      = pd.DataFrame(df__.apply(lambda x :  x['bms_02_100ms_2_xev_battcurrval'], axis = 1 ))       
    df__['temp']      = pd.DataFrame(df__.apply(lambda x : (x['bms_02_100ms_2_xev_maxbatttempval']+x['bms_02_100ms_2_xev_minbatttempval'])/2, axis = 1 ))       
    df__['soc']       = pd.DataFrame(df__.apply(lambda x : (x['bms_01_100ms_2_xev_socval']+x['bms_01_100ms_2_xev_socdecval']), axis = 1 ))       
    
    num_trip = len(df__['index'].unique())
    
    print(f'vin: {vin_num} / # of trip: {num_trip}')
    
    for k in df__['index'].unique():
    
        df_trip = df__.loc[df__['index']==k,:].drop_duplicates().reset_index(drop = True)
    
        idx_trip = df_trip['index'].iloc[0]
        len_trip = len(df_trip)
        
        print(f'idx_trip: {idx_trip} / len_trip: {len_trip}')
        
        
        if len(df_trip) > TIME_STEP:
            
            for i in range(len(df_trip)-TIME_STEP+1):
                window = df_trip.iloc[i:(i+TIME_STEP),:]
                window = window[['vdiff','curr', 'temp', 'soc']]
                window = window.values.tolist()

                test_list4_1.append(window)
                 

In [ ]:
test_list5_1 = []
for (vin_num,case) in [[5,'BEFORE']]:
    
    df__ = pd.read_csv(f"./vin_{vin_num:02d}_2021_{case}_SORT.csv")
    
    df__['vdiff']     = pd.DataFrame(df__.apply(lambda x : (x['bms_26_500ms_2_xev_bms_vmax_dec']-x['bms_26_500ms_2_xev_bms_vmin_dec']), axis = 1 ))   
    df__['curr']      = pd.DataFrame(df__.apply(lambda x :  x['bms_02_100ms_2_xev_battcurrval'], axis = 1 ))       
    df__['temp']      = pd.DataFrame(df__.apply(lambda x : (x['bms_02_100ms_2_xev_maxbatttempval']+x['bms_02_100ms_2_xev_minbatttempval'])/2, axis = 1 ))       
    df__['soc']       = pd.DataFrame(df__.apply(lambda x : (x['bms_01_100ms_2_xev_socval']+x['bms_01_100ms_2_xev_socdecval']), axis = 1 ))       
    
    num_trip = len(df__['index'].unique())
    
    print(f'vin: {vin_num} / # of trip: {num_trip}')
    
    for k in df__['index'].unique():
    
        df_trip = df__.loc[df__['index']==k,:].drop_duplicates().reset_index(drop = True)
    
        idx_trip = df_trip['index'].iloc[0]
        len_trip = len(df_trip)
        
        print(f'idx_trip: {idx_trip} / len_trip: {len_trip}')
        
        
        if len(df_trip) > TIME_STEP:
            
            for i in range(len(df_trip)-TIME_STEP+1):
                window = df_trip.iloc[i:(i+TIME_STEP),:]
                window = window[['vdiff','curr', 'temp', 'soc']]
                window = window.values.tolist()

                test_list5_1.append(window)
                 

In [ ]:
test_list6_1 = []
for (vin_num,case) in [[6,'BEFORE']]:
    
    df__ = pd.read_csv(f"./vin_{vin_num:02d}_2021_{case}_SORT.csv")
    
    df__['vdiff']     = pd.DataFrame(df__.apply(lambda x : (x['bms_26_500ms_2_xev_bms_vmax_dec']-x['bms_26_500ms_2_xev_bms_vmin_dec']), axis = 1 ))   
    df__['curr']      = pd.DataFrame(df__.apply(lambda x :  x['bms_02_100ms_2_xev_battcurrval'], axis = 1 ))       
    df__['temp']      = pd.DataFrame(df__.apply(lambda x : (x['bms_02_100ms_2_xev_maxbatttempval']+x['bms_02_100ms_2_xev_minbatttempval'])/2, axis = 1 ))       
    df__['soc']       = pd.DataFrame(df__.apply(lambda x : (x['bms_01_100ms_2_xev_socval']+x['bms_01_100ms_2_xev_socdecval']), axis = 1 ))       
    
    num_trip = len(df__['index'].unique())
    
    print(f'vin: {vin_num} / # of trip: {num_trip}')
    
    for k in df__['index'].unique():
    
        df_trip = df__.loc[df__['index']==k,:].drop_duplicates().reset_index(drop = True)
    
        idx_trip = df_trip['index'].iloc[0]
        len_trip = len(df_trip)
        
        print(f'idx_trip: {idx_trip} / len_trip: {len_trip}')
        
        
        if len(df_trip) > TIME_STEP:
            
            for i in range(len(df_trip)-TIME_STEP+1):
                window = df_trip.iloc[i:(i+TIME_STEP),:]
                window = window[['vdiff','curr', 'temp', 'soc']]
                window = window.values.tolist()

                test_list6_1.append(window)

In [ ]:
with open( "vin_01_2021_BEFORE_SORT_data_list_data", "wb" ) as file:
    pickle.dump( test_list1_1, file)
with open( "vin_01_2021_AFTER_SORT_data_list_data", "wb" ) as file:
    pickle.dump( test_list1_2, file)    
with open( "vin_02_2021_BEFORE_SORT_data_list_data", "wb" ) as file:
    pickle.dump( test_list2_1, file)
with open( "vin_02_2021_AFTER_SORT_data_list_data", "wb" ) as file:
    pickle.dump( test_list2_2, file)
with open( "vin_04_2021_BEFORE_SORT_data_list_data", "wb" ) as file:
    pickle.dump( test_list4_1, file)
with open( "vin_05_2021_BEFORE_SORT_data_list_data", "wb" ) as file:
    pickle.dump( test_list5_1, file)   
with open( "vin_06_2021_BEFORE_SORT_data_list_data", "wb" ) as file:
    pickle.dump( test_list6_1, file)

In [ ]:
train_data, val_data = train_test_split(train_list, test_size=0.2)
print(train_data.shape,val_data.shape)

# 정규화는 학습데이터에 대하여 진행이 되므로 train_data를 numpy 형태로 만들어줍니다.
minmax_arr=np.array(train_data)
# MinMaxScaler()를 minmax에 선언을 해줍니다.
minmax = MinMaxScaler()
# 만들어준 train data의 배열을 4개의 columns로 구성이 될 수 있도록 구조를 변형해준 후 minmax.fit을 통해 최대 최소 정규화에 필요한 minmax 값을 학습해줍니다.
minmax.fit(minmax_arr.reshape(-1,4))

print("min값 : ",minmax.data_min_, "max값 : ",minmax.data_max_)

# 학습된 minmax를 각 데이터셋에 적용하기 위해 minmax.transform 함수를 사용해줍니다.
# 4개의 columns으로 구성이 되어야 적용이 되므로 구조를 바꿔주었다가, LSTM 구조에 들어갈 수 있도록 (배치개수, sequence_length, features) 3차원 구조로 다시 바꾸어줍니다.
train_minmax=(minmax.transform(train_data.reshape(-1,4))).reshape(-1,60,4)
valid_minmax=(minmax.transform(val_data.reshape(-1,4))).reshape(-1,60,4)

# 최종적으로 학습에 사용될 수 있도록 텐서화 시켜줍니다.
train_tensor = Variable(torch.Tensor(train_minmax))
valid_tensor = Variable(torch.Tensor(valid_minmax))


with open( "train_tensor", "wb" ) as file:
    pickle.dump( train_tensor, file)
with open( "valid_tensor", "wb" ) as file:
    pickle.dump( valid_tensor, file)    

# list 데이터셋이 들어오면 numpy 형태로 바꾸어주고, reshape를 통해 4개의 columns 구조로 바꾸어 정규화 시켜준 후 학습에 사용이 될 수 있도록 3차원 구조로 다시바꿔줍니다.
# 최종적으로 학습에 사용될 수 있도록 텐서화 시켜준 것을 출력해주는 함수입니다.
def preprocessing(data, scaler):
    data_arr=np.array(data)
    data_minmax=(scaler.transform(data_arr.reshape(-1,4))).reshape(-1,60,4)
    return Variable(torch.Tensor(data_minmax))

# 학습데이터 전처리
train42_tensor =  preprocessing(train_list4_2, minmax)
train52_tensor =  preprocessing(train_list5_2, minmax)
train62_tensor =  preprocessing(train_list6_2, minmax)

# 테스트데이터 전처리
test11_tensor  =  preprocessing(test_list1_1, minmax)
test12_tensor  =  preprocessing(test_list1_2, minmax)
test21_tensor  =  preprocessing(test_list2_1, minmax)
test22_tensor  =  preprocessing(test_list2_2, minmax)
test41_tensor  =  preprocessing(test_list4_1, minmax)
test51_tensor  =  preprocessing(test_list5_1, minmax)
test61_tensor  =  preprocessing(test_list6_1, minmax)

In [ ]:
# pickle 모듈을 이용해 평가에 사용할 데이터 저장
with open( "vin_01_2021_BEFORE_SORT_data_tensor", "wb" ) as file:
    pickle.dump( test11_tensor, file)
with open( "vin_01_2021_AFTER_SORT_data_tensor", "wb" ) as file:
    pickle.dump( test12_tensor, file)
with open( "vin_02_2021_BEFORE_SORT_data_tensor", "wb" ) as file:
    pickle.dump( test21_tensor, file)
with open( "vin_02_2021_AFTER_SORT_data_tensor", "wb" ) as file:
    pickle.dump( test22_tensor, file)
with open( "vin_04_2021_BEFORE_SORT_data_tensor", "wb" ) as file:
    pickle.dump( test41_tensor, file)
with open( "vin_04_2021_AFTER_SORT_data_tensor", "wb" ) as file:
    pickle.dump( train42_tensor, file)
with open( "vin_05_2021_BEFORE_SORT_data_tensor", "wb" ) as file:
    pickle.dump( test51_tensor, file)
with open( "vin_05_2021_AFTER_SORT_data_tensor", "wb" ) as file:
    pickle.dump( train52_tensor, file)   
with open( "vin_06_2021_BEFORE_SORT_data_tensor", "wb" ) as file:
    pickle.dump( test61_tensor, file)
with open( "vin_06_2021_AFTER_SORT_data_tensor", "wb" ) as file:
    pickle.dump( train62_tensor, file)

In [ ]:
# 사용한 4개의 변수
n_features=4

# 배치사이즈 16
batch_size=16

# sequence length 60
seq_len=60

In [ ]:
# 인코더 설계
class Encoder(nn.Module):
    # input으로 사용 변수 개수와 latent vector의 차원을 받음
    def __init__(self, n_features, embedding_dim=64):
        super(Encoder, self).__init__()
        # class에서 사용될 변수들을 init부분에서 선언을 해줍니다.
        self.n_features = n_features
        # embedding_dim은 그대로 input을 받아오고, hidden_dim은 embedding의 절반인 32로 설정하였습니다.
        self.embedding_dim, self.hidden_dim = embedding_dim, 32
        # 인코더를 lstm으로 구성하였습니다.
        # 차원이 4 -> 32 -> 64로 진행됩니다.
        self.lstm1 = nn.LSTM(input_size=n_features,hidden_size=self.hidden_dim,num_layers=1,batch_first=True)
        self.lstm2 = nn.LSTM(input_size=self.hidden_dim,hidden_size=embedding_dim,num_layers=1,batch_first=True)
        
    # forward 함수에서 lstm층을 연결해줍니다.
    def forward(self, x):
        x, (_, _) = self.lstm1(x) 
        x, (hidden_n, _) = self.lstm2(x)
        # 마지막으로 나오는 벡터가 latent vector가 되는데, unsqueeze를 시켜주어 3차원 형태로 만들어 줍니다.
        return x[:,-1,:].unsqueeze(1)

# 디코더 설계
class Decoder(nn.Module):
    # input으로 seq_len, input dim=embedding_dim, 변수 개수를 받아옵니다.
    def __init__(self, seq_len, input_dim=32, n_features=4):
        super(Decoder, self).__init__()
        # encoder와 마찬가지로 사용할 변수들을 선언해주고 lstm을 구성해줍니다.
        self.seq_len, self.input_dim = seq_len, input_dim
        self.hidden_dim, self.n_features = 2 * input_dim, n_features
        # lstm층의 차원을 설정해주는데, 64 -> 32 -> 4로 진행되도록 설정해줍니다.
        self.lstm1 = nn.LSTM(input_size=input_dim,hidden_size=input_dim,num_layers=1,batch_first=True)
        self.lstm2 = nn.LSTM(input_size=input_dim,hidden_size=self.hidden_dim,num_layers=1,batch_first=True)
        # 최종적으로 나오는 output layer는 linear 함수를 사용하여 결과값을 도출해줍니다.
        self.output_layer = nn.Linear(self.hidden_dim, n_features)
    
    # forward 함수에서 lstm층을 연결해줍니다.
    def forward(self, x):
        # seq_len 개수 만큼 latent vecotr를 .repeat을 통해 복사해줍니다.
        x = x.repeat(1, seq_len, 1)
        x, (hidden_n, cell_n) = self.lstm1(x)
        x, (hidden_n, cell_n) = self.lstm2(x)
        return self.output_layer(x)

In [ ]:
# 최종적으로 encoder와 decoder를 연결해줍니다.
class LstmAutoencoder(nn.Module):
    def __init__(self, seq_len, n_features, embedding_dim=32):
        super(LstmAutoencoder, self).__init__()
        self.encoder = Encoder(n_features, embedding_dim).to(device)
        self.decoder = Decoder(seq_len, embedding_dim, n_features).to(device)
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
# lstm autoencoder모델을 선언해주고 gpu로 보내줍니다.
model = LstmAutoencoder(seq_len, n_features)
model = model.to(device)

In [ ]:
def train_model(model, train_dataset, val_dataset, n_epochs, batch_size):
    # 최적화를 시키는 부분으로 torch.optim 모듈에 있는 Adam 알고리즘을 사용하였습니다.
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    # 오차를 구하는 기준 함수로 예측한 값의 절대값 차이를 모두 더해 loss로 정의한 L1 loss를 사용하였습니다. 
    criterion = nn.L1Loss().to(device)
    # 오차율을 각 epoch별로 담을 history를 선언해줍니다.
    history = dict(train=[], val=[])
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 10000
    print("start!")
    
    #데이터로더로 학습데이터셋을 배치사이즈 만큼씩 보낼 수 있도록 재구성해줍니다.
    tr_dataloader = DataLoader(train_dataset, batch_size=batch_size, 
                               shuffle=False,num_workers=0)
    va_dataloader = DataLoader(val_dataset, batch_size=len(val_dataset),
                               shuffle=False,num_workers=0)
    
    # 지정한 epoch만큼 for 문을 돌려 학습을 진행합니다.
    for epoch in range(1, n_epochs + 1):
        # 모델이 학습을 진행합니다.
        model = model.train()
        train_losses = []
        for batch_idx, batch_x in enumerate(tr_dataloader):
            # 변화도를 초기화 해줍니다.
            optimizer.zero_grad()
            # 배치 사이즈 만큼씩 데이터 전송
            batch_x_tensor = batch_x.to(device)
            seq_pred = model(batch_x_tensor)
            # 오차를 계산하고 역전파시켜서 나온 파라미터 값을 갱신해줍니다.
            loss = criterion(seq_pred, batch_x_tensor)
            loss.backward()
            optimizer.step()
            # train loss값을 기록해줍니다.
            train_losses.append(loss.item())
        val_losses = []
        #평가 모드로 변경해줍니다.
        model = model.eval()
        #평가후, 오차를 계산해줍니다.
        with torch.no_grad():
            va_x  =next(va_dataloader.__iter__())
            va_x_tensor = va_x.to(device)
            seq_pred = model(va_x_tensor)
            loss = criterion(seq_pred, va_x_tensor)
            val_losses.append(loss.item())
        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)
        history['train'].append(train_loss)
        history['val'].append(val_loss)
        # 평가 오차가 작은 것을 계속 업데이트 해줍니다.
        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
        print(f'Epoch {epoch}: train loss {train_loss} val loss {val_loss}')
    #가장 좋은 모델 파라미터를 가져와줍니다.
    model.load_state_dict(best_model_wts)
    return model.eval(), history

In [ ]:
# 모델을 사용하여 예측을 해주는 함수입니다.
def predict(model, dataset,batch_size=1):
    predictions, losses = [], []
    criterion = nn.L1Loss().to(device)
    # 테스트의 경우 배치사이즈를 1로하여 하나씩 평가될 수 있도록 합니다.
    dataloader_ae = DataLoader(dataset, 
                               batch_size=batch_size,
                               shuffle=False,num_workers=0)
    
    with torch.no_grad():
        # 모델을 평가모드로 바꾸어줍니다.
        model = model.eval()
        if batch_size == len(dataset) :
            seq_true  =next(dataloader_ae.__iter__())
            seq_true = seq_true.to(device)
            seq_pred = model(seq_true)
            loss = criterion(seq_pred, seq_true)
            predictions.append(seq_pred.cpu().numpy().flatten())
            losses.append(loss.item())
        else :
            for idx , seq_true in enumerate(dataloader_ae):
                seq_true = seq_true.to(device)
                seq_pred = model(seq_true)
                loss = criterion(seq_pred, seq_true)
                predictions.append(seq_pred.cpu().numpy().flatten())
                losses.append(loss.item())
    return predictions, losses

In [ ]:
trainpred, history = train_model(model, train_tensor, valid_tensor , n_epochs = 200, batch_size=16)


In [ ]:
# plot the training losses
fig, ax = plt.subplots(figsize=(14, 6), dpi=80)
ax.plot(history['train'], 'b', label='Train', linewidth=2)
ax.plot(history['val'], 'r', label='Validation', linewidth=2)
ax.set_title('Model loss', fontsize=16)
ax.set_ylabel('Loss (mae)')
ax.set_xlabel('Epoch')
ax.legend(loc='upper right')
plt.show()

In [ ]:
#모델 저장, 예측값 저장 부분
torch.save(model, '8_27.pth')

In [ ]:
X_pred4_2,losses4_2 = predict(model,train42_tensor)
X_pred5_2,losses5_2 = predict(model,train52_tensor)
X_pred6_2,losses6_2 = predict(model,train62_tensor)

In [ ]:
X_pred1_1,losses1_1 = predict(model,test11_tensor)
X_pred1_2,losses1_2 = predict(model,test12_tensor)
X_pred2_1,losses2_1 = predict(model,test21_tensor)
X_pred2_2,losses2_2 = predict(model,test22_tensor)
X_pred4_1,losses4_1 = predict(model,test41_tensor)
X_pred5_1,losses5_1 = predict(model,test51_tensor)
X_pred6_1,losses6_1 = predict(model,test61_tensor)

In [ ]:
def plot_voltage_2dataset(original_data, pred_data):
    # 입력된 두개의 데이터셋을 numpy 형태로 바꾸어주고 3차원 형태로 바꾸어줍니다. 
    reshaped_org  = original_data.reshape(-1,60,4)
    pred_data=np.array(pred_data)
    reshaped_pred = pred_data.reshape(-1,60,4)
    
    mean_data=np.empty((reshaped_org.shape[0] ,1), object)
    #윈도우 별로 평균 처리 해줍니다.
    for i in range(reshaped_org.shape[0]):
        mean_data[i]=reshaped_org[i,:,0].mean()
        
        
    mean_pred=np.empty((reshaped_pred.shape[0] ,1), object)
    #윈도우 별로 평균 처리 해줍니다.
    for i in range(reshaped_pred.shape[0]):
        mean_pred[i]=reshaped_pred[i,:,0].mean()

    
    plt.figure(figsize=(12,6))
    plt.plot(mean_data,color='red')
    plt.plot(mean_pred)
    plt.show()

In [ ]:
def plot_voltage(data):
    # 입력된 데이터셋을 numpy 형태로 바꾸어주고 3차원 형태로 바꾸어줍니다. 
    data_arr=np.array(data)
    reshaped_data_arr = data_arr.reshape(-1,60,4)
    
    mean_data=np.empty((reshaped_data_arr.shape[0] ,1), object)
    #윈도우 별로 평균 처리 해줍니다.
    for i in range(reshaped_data_arr.shape[0]):
        mean_data[i]=reshaped_data_arr[i,:,0].mean()
    
    plt.figure(figsize=(12,6))
    plt.plot(mean_data)
    plt.show()

In [ ]:
def plot_voltage_loss(original_data, pred_data):
    # 입력된 두개의 데이터셋을 numpy 형태로 바꾸어주고 3차원 형태로 바꾸어줍니다. 
    reshaped_org  = original_data.reshape(-1,60,4)
    pred_data=np.array(pred_data)
    reshaped_pred = pred_data.reshape(-1,60,4)
    
    mean_data=np.empty((reshaped_org.shape[0] ,1), object)
    # 윈도우별로 평균 처리해줍니다.
    for i in range(reshaped_org.shape[0]):
        mean_data[i]=reshaped_org[i,:,0].mean()
    mean_data= pd.DataFrame(mean_data, columns=['Loss'])


    mean_pred=np.empty((reshaped_pred.shape[0] ,1), object)
    # 윈도우별로 평균 처리해줍니다.
    for i in range(reshaped_pred.shape[0]):
        mean_pred[i]=reshaped_pred[i,:,0].mean()
    mean_pred= pd.DataFrame(mean_pred, columns=['Loss'])

    mean_loss=np.zeros(shape=(reshaped_org.shape[0],1))
    mean_loss= pd.DataFrame(mean_loss, columns=['Loss'])

    loss=[]
    # 예측값과 원본값의 차이를 절대값 씌워주어 오차를 계산해줍니다.
    for i in range(reshaped_org.shape[0]):
        temp_loss=abs(mean_pred['Loss'][i]-mean_data['Loss'][i])
        loss.append(temp_loss)
    mean_loss['Loss'] = loss
    mean_loss_arr=mean_loss['Loss'].to_numpy()
    
    plt.figure(figsize=(12,6))
    plt.plot(mean_loss_arr)
    plt.show()

In [ ]:
def loss_plot(data):
    # 입력된 데이터를 numpy형태로 바꾸어주고 plot해줍니다.
    loss_arr=np.array(data)
    plt.figure(figsize=(12,6))
    plt.plot(loss_arr)
    plt.show()
def denormalize(data, pred_data, scaler):
    # 원본 데이터와 예측된 데이터, 사용된 scaler를 입력 받습니다.
    # 두개의 데이터셋을 numpy 형태로 바꾸어주고 비정규화를 위해 4개의 columns 구조로 변형해줍니다.
    data_arr=np.array(data)
    pred_arr=np.array(pred_data)
    data_arr_reshape=data_arr.reshape(-1,4)
    pred_arr_reshape=pred_arr.reshape(-1,4)
    # inverse transform함수를 사용하여 비정규화 시켜줍니다.
    denormal_data=scaler.inverse_transform(data_arr_reshape)
    denormal_pred=scaler.inverse_transform(pred_arr_reshape)
    
    return denormal_data, denormal_pred

In [ ]:
def plot_mean_total_loss(original_data, pred_data):
    # 각 데이터셋을 3차원으로 만들어줍니다.
    reshaped_org  = original_data.reshape(-1,60,4)
    reshaped_pred = pred_data.reshape(-1,60,4)
    
    mean_data=np.empty((reshaped_org.shape[0] ,4), object)
    # 윈도우 평균처리
    for i in range(reshaped_org.shape[0]):
        for j in range(4):
            mean_data[i][j]=reshaped_org[i,:,j].mean()
    mean_pred=np.empty((reshaped_pred.shape[0] ,4), object)
    # 윈도우 평균처리
    for i in range(reshaped_pred.shape[0]):
        for j in range(4):
            mean_pred[i][j]=reshaped_pred[i,:,j].mean()
    # 데이터 계산의 용이함을 위해 pandas dataframe 형태로 바꾸어줍니다.
    mean_data= pd.DataFrame(mean_data, columns=['vdiff','curr','temp','soc'])
    mean_pred= pd.DataFrame(mean_pred, columns=['vdiff','curr','temp','soc'])

    
    mean_loss=np.zeros(shape=(reshaped_org.shape[0],1))
    mean_loss= pd.DataFrame(mean_loss, columns=['Loss'])

    loss=[]
    
    # 각 columns 별로 차이값을 계산하고 평균을 구해주어 최종적인 loss값을 계산해줍니다.
    for i in range(reshaped_org.shape[0]):
        temp_loss=abs(mean_pred['vdiff'][i]-mean_data['vdiff'][i])+abs(mean_pred['curr'][i]-mean_data['curr'][i])+abs(mean_pred['temp'][i]-mean_data['temp'][i])+abs(mean_pred['soc'][i]-mean_data['soc'][i])
        loss.append(temp_loss/4)
    mean_loss['Loss'] = loss
    mean_loss_arr=mean_loss['Loss'].to_numpy()

    plt.figure(figsize=(12,6))
    plt.plot(mean_loss_arr)
    plt.show()

In [ ]:
# train set plot
plot_voltage(train42_tensor)
plot_voltage(train52_tensor)
plot_voltage(train62_tensor)
# test set plot
plot_voltage(test11_tensor)
plot_voltage(test12_tensor)
plot_voltage(test21_tensor)
plot_voltage(test22_tensor)
plot_voltage(test41_tensor)
plot_voltage(test51_tensor)
plot_voltage(test61_tensor)

In [ ]:
# train set plot
plot_voltage(X_pred4_2)
plot_voltage(X_pred5_2)
plot_voltage(X_pred6_2)
# test set plot
plot_voltage(X_pred1_1)
plot_voltage(X_pred1_2)
plot_voltage(X_pred2_1)
plot_voltage(X_pred2_2)
plot_voltage(X_pred4_1)
plot_voltage(X_pred5_1)
plot_voltage(X_pred6_1)

In [ ]:
# train set plot
plot_voltage_2(train42_tensor,X_pred4_2)
plot_voltage_2(train52_tensor,X_pred5_2)
plot_voltage_2(train62_tensor,X_pred6_2)
# test set plot
plot_voltage_2(test11_tensor,X_pred1_1)
plot_voltage_2(test12_tensor,X_pred1_2)
plot_voltage_2(test21_tensor,X_pred2_1)
plot_voltage_2(test22_tensor,X_pred2_2)
plot_voltage_2(test41_tensor,X_pred4_1)
plot_voltage_2(test51_tensor,X_pred5_1)
plot_voltage_2(test61_tensor,X_pred6_1)

In [ ]:
# train set plot
plot_voltage_loss(train42_tensor,X_pred4_2)
plot_voltage_loss(train52_tensor,X_pred5_2)
plot_voltage_loss(train62_tensor,X_pred6_2)
# test set plot
plot_voltage_loss(test11_tensor,X_pred1_1)
plot_voltage_loss(test12_tensor,X_pred1_2)
plot_voltage_loss(test21_tensor,X_pred2_1)
plot_voltage_loss(test22_tensor,X_pred2_2)
plot_voltage_loss(test41_tensor,X_pred4_1)
plot_voltage_loss(test51_tensor,X_pred5_1)
plot_voltage_loss(test61_tensor,X_pred6_1)